In [1]:

import pandas as pd
from PIL import Image
import pytesseract
import os
import numpy as np
from PIL import Image, ImageEnhance
import re
import os
import os, json, joblib
from datetime import datetime
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

print(np.__version__)

2.1.3


In [2]:
def clean_text(raw_text):
    # Collapse whitespace
    text = re.sub(r"\s+", " ", raw_text).strip()

    # Remove common header/footer artifacts
    text = re.sub(r"Page\s*\d+|\d+\s*/\s*\d+", "", text, flags=re.IGNORECASE)

    # Strict zero replacement (only in CAPS words)
    text = re.sub(r"\b([A-Z]*?)0([A-Z]*?)\b", lambda m: m.group(0).replace("0", "O"), text)
    
    # Looser one replacement (fixes Th1s → This)
    text = re.sub(r"\b(\w*?)1(\w*?)\b", lambda m: m.group(0).replace("1", "l"), text)

    # Lowercase for consistency
    text = text.lower()

    # Optional: remove punctuation
    text = re.sub(r"[^\w\s]", "", text)

    return text


In [3]:
def preprocess_image(img):
    img_rgb = img.convert("RGB")
    enhancer = ImageEnhance.Contrast(img_rgb)
    img_contrast = enhancer.enhance(2.0)
    return img_contrast.convert("L").point(lambda x: 0 if x < 128 else 255, "1")

In [4]:
def compare_distributions(full_df, list_of_dfs, column, tolerance=1):
    true_pct = full_df[column].value_counts(normalize=True)*100
    labels = true_pct.index
    results = {}
    for name, df in list_of_dfs:
        df_pct = df[column].value_counts(normalize=True)*100
        df_pct = df_pct.reindex(labels, fill_value=0,)
        diff = (np.abs(true_pct - df_pct))
        bad = diff[diff > tolerance]
        ok = (diff<=tolerance).all()
        if ok:
            print (f"✅ Distributions are similar full vs {name}")
            
        else:
            print (f"🚨 Distributions from {name} are more than {tolerance}% "
                    f"different from full dataset {bad.index} with differences {bad.values}")
        results[name] = {'ok': bool(ok),
                        'max_drift': float(diff.max()), 
                        'bad_labels': bad.index.tolist()}
    return results

In [6]:
from pathlib import Path
print("CWD:", Path.cwd())
img_dir = Path("data/raw/images/Training")
print("Looking for:", img_dir.resolve())
print("Exists?", img_dir.exists())

CWD: C:\Users\abajp\PycharmProjects\BofAOCRProject\notebooks
Looking for: C:\Users\abajp\PycharmProjects\BofAOCRProject\notebooks\data\raw\images\Training
Exists? False


In [7]:
from pathlib import Path
import os, pandas as pd
from PIL import Image
import pytesseract

file_types = ['budget', 'email', 'letter', 'invoice']
cols = ['filename', 'type', 'text']

img_dir = Path(r"C:\Users\abajp\PycharmProjects\BofAOCRProject\data\raw\images\Training")
processed_dir = Path(r"C:\Users\abajp\PycharmProjects\BofAOCRProject\data\processed")
processed_dir.mkdir(parents=True, exist_ok=True)

# img_dir       = Path("data/raw/images/Training")
# processed_dir = Path("data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)
out_path = processed_dir / "output.csv"

# load processed filenames once (fast: read only that column)
processed = set()
if out_path.exists():
    processed = set(pd.read_csv(out_path, usecols=['filename'])['filename'])

for name in os.listdir(img_dir):
    path = img_dir / name
    if not path.is_file():
        continue

    match = next((w for w in file_types if w in name.lower()), None)
    if not match:
        continue
    if name in processed:
        continue

    with Image.open(path) as img:
        pre_img = preprocess_image(img)
        text = clean_text(pytesseract.image_to_string(pre_img))

    # append only the new row
    new_df = pd.DataFrame([[name, match, text]], columns=cols)
    new_df.to_csv(out_path, mode='a', index=False, header=not out_path.exists())
    processed.add(name)

In [24]:
files = os.listdir("data/raw/images/Training")
file_types = ['budget', 'email', 'letter', 'invoice']
cols = ['filename', 'type', 'text']
from pathlib import Path
img_dir = Path("data/raw/images/Training")
processed_dir = Path("data/processed")

contents = []
file_dict = {}
output_file = "output.csv"

if not os.path.exists(output_file):
    print("🚨 CSV does not exist, creating blank one...")
    df = pd.DataFrame(columns=cols)
    df.to_csv(output_file, index=False)  # Create blank CSV with header
else:
    print("✅ CSV exists already")
    df = pd.read_csv(output_file)

for name in os.listdir(img_dir):
    path = img_dir/name
    match = next((word for word in file_types if word.lower() in name.lower()), None)
    if match:
        print(f"{name}:{match}")
    processed_files = set(df['filename'])
    if path in processed_files:
        print(f"{name} already in df")
        continue
    file_dict.update({path:match})
    with Image.open(path) as img:
        pre_img = preprocess_image(img)
        text = pytesseract.image_to_string(pre_img)
        # text = pytesseract.image_to_string(Image.open(f"Training_half/{i}"))
        text = clean_text(text)   
        # contents.append(text)
        new_df = pd.DataFrame([[path, match, text]], columns = cols)
        df = pd.concat([df, new_df], ignore_index=True)
        if not os.path.exists("output.csv"):
            df.to_csv(f"{processed_dir}/output.csv", index=False)  # Write with header
        else:
            df.to_csv(f"{processed_dir}/output.csv", mode="a", index=False, header=False)
        


✅ CSV exists already
advertisement0.tif already in df
advertisement1.tif already in df
advertisement10.tif already in df
advertisement100.tif already in df
advertisement1000.tif already in df
advertisement10000.tif already in df
advertisement10001.tif already in df
advertisement10002.tif already in df
advertisement10003.tif already in df
advertisement10004.tif already in df
advertisement10005.tif already in df
advertisement10006.tif already in df
advertisement10007.tif already in df
advertisement10008.tif already in df
advertisement10009.tif already in df
advertisement1001.tif already in df
advertisement10010.tif already in df
advertisement10011.tif already in df
advertisement10012.tif already in df
advertisement10013.tif already in df
advertisement10014.tif already in df
advertisement10015.tif already in df
advertisement10016.tif already in df
advertisement10017.tif already in df
advertisement10018.tif already in df
advertisement10019.tif already in df
advertisement1002.tif already in

KeyboardInterrupt: 

In [19]:
# Save a quick look at the nulls
nulls = df[df.isna().any(axis=1)]
nulls.to_csv("null_rows_snapshot.csv", index=False)

# Drop nulls & empty strings in core columns
df = df.dropna(subset=["text", "type"])
df = df[df["text"].str.strip() != ""]

In [20]:
from sklearn.model_selection import train_test_split
train_val, test = train_test_split(df, test_size=0.1, random_state=42, stratify=df['type'])
train, val = train_test_split(train_val, test_size=0.09, random_state=42, stratify=train_val['type'])

In [21]:
train_val_pct = train_val.value_counts('type')/len(train_val)*100
test_val_pct = test.value_counts('type')/len(test)*100
diff = train_val_pct - test_val_pct
diff

type
budget     0.000370
letter     0.004541
email      0.001745
invoice   -0.006655
Name: count, dtype: float64

In [39]:
results = compare_distributions(df, [('train', train), ('val', val), ('test', test)], 'type')
results

✅ Distributions are similar full vs train
✅ Distributions are similar full vs val
✅ Distributions are similar full vs test


{'train': {'ok': True, 'max_drift': 0.0012992425166302723, 'bad_labels': []},
 'val': {'ok': True, 'max_drift': 0.007846731365585669, 'bad_labels': []},
 'test': {'ok': True, 'max_drift': 0.005989756978138416, 'bad_labels': []}}

In [40]:
train.to_csv("csv_files/train.csv", index=False)
val.to_csv("csv_files/validation.csv", index=False)
test.to_csv("csv_files/test.csv", index=False)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=50000, min_df=2, sublinear_tf=True)
X_train = vectorizer.fit_transform(train['text'].astype(str))
X_va = vectorizer.transform(val['text'].astype(str))

In [47]:
codes = {'letter':0, 'budget':1, 'invoice':2, 'email':3}

y_train = train['type']
y_va = val['type']
y_train = y_train.map(codes)
y_va =  y_va.map(codes)

In [48]:
# train logreg model and show performance of model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

clf = LogisticRegression(solver='liblinear', max_iter=2000, random_state=42)
clf.fit(X_train, y_train)

preds = clf.predict(X_va)
print(classification_report(y_va, preds, target_names=codes.keys()))

              precision    recall  f1-score   support

      letter       0.92      0.92      0.92      1617
      budget       0.84      0.91      0.87      1624
     invoice       0.89      0.85      0.87      1598
       email       0.97      0.91      0.94      1613

    accuracy                           0.90      6452
   macro avg       0.90      0.90      0.90      6452
weighted avg       0.90      0.90      0.90      6452

